# Elementos da versão

Antes mesmo de falar do projeto, ao longo do desenvolvimento, vamos explicitando qual o objetivo de cada versão de trabalho do arquivo principal.

Tivemos problemas de análise dos resultados da v1. Os dados de mortalidade de doenças evitáveis em 2022 por município estão muito altos e fora do padrão esperado.

O objetivo desta v.2 é criar um dataset diferente, mais básico que permite que aprendamos com os erros e acertos, verificando melhor cada etapa de trabalho.

O dataset desta v.2 deverá ter as seguintes características:

* setting: Brasil [poderemos encontrar um sufixo a Brasil que permita a sua subclassificação e carga deste dataset junto a outros]
* dimension: CID evitáveis
* subgroup: municípios
* estimate: mort-avoid por município
* setting-average: mort-avoid por Brasil

# Projeto de Análise de Desigualdade em Saúde no modelo HEAT/OMS
## Bloco 2025/1 = análise do sistema de mortalidade do SUS com agregação por unidades federativas




#### Aspectos gerais e participantes

#### Objetivos

# Merge de dados de CID e População no arquivo principal

### Merge de dados de CID

In [2]:
ocde_evit_temp = pd.read_csv('downloads_outros/cid10_evit_ocde.csv')
ocde_evit_temp.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 346 entries, 0 to 345
Data columns (total 8 columns):
 #   Column                   Non-Null Count  Dtype 
---  ------                   --------------  ----- 
 0   avoid_flag               346 non-null    object
 1   prevent_flag             189 non-null    object
 2   treat_flag               184 non-null    object
 3   Group                    346 non-null    object
 4   Causes of deaths         346 non-null    object
 5   Rationale for inclusion  346 non-null    object
 6   Range                    346 non-null    object
 7   cid_pai                  346 non-null    object
dtypes: object(8)
memory usage: 21.8+ KB


In [3]:
# Gerar nova tabela com valores únicos de CAUSABAS
dobr_evit_merged_temp = pd.read_parquet('files_clean/dobr_10_19a22.parquet')
causas_unicas = dobr_evit_merged_temp['CAUSABAS'].unique()

# Converter para DataFrame
df_causas_unicas = pd.DataFrame(causas_unicas, columns=['CAUSABAS'])

# Exibir as primeiras linhas do DataFrame
df_causas_unicas.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11895 entries, 0 to 11894
Data columns (total 1 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   CAUSABAS  11895 non-null  object
dtypes: object(1)
memory usage: 93.1+ KB


In [4]:
# Fazer merge da lista de causas_unicas com a lista de causas evitáveis

df_a = df_causas_unicas
column_a = 'CAUSABAS'
df_b = ocde_evit_temp
column_b = 'cid_pai'
column_return = 'avoid_flag'



# Função para verificar a correspondência e retornar a classificação
def verificar_correspondencia(row):
    for index, row_b in df_b.iterrows():
        if row[column_a].startswith(row_b[column_b]):
            return row_b[column_return]
    return 'causa não evitável'

# Aplicando a função a cada linha do DataFrame A
df_a['classificacao'] = df_a.apply(verificar_correspondencia, axis=1)

print(tabulate(df_a.head()))

-  ----  -------------------
0  I219  Avoidable mortality
1  I10   Avoidable mortality
2  X700  Avoidable mortality
3  C539  causa não evitável
4  V220  Avoidable mortality
-  ----  -------------------


In [68]:
# Merge dobr_merged_pop_clean and df_a by CAUSABAS
dobr_evit_merged_temp01 = pd.merge(dobr_evit_merged_temp, df_a, on='CAUSABAS', how='left')

# mostrar dados gerais da tabela após transformações
tab = dobr_evit_merged_temp01
print(f'Registro da tabela : {tab.shape[0]}')
print(tabulate(tab.head(), headers='keys', tablefmt='psql'))

Registro da tabela : 4446922
+----+------------+-------------+-------------+---------------------+
|    | CAUSABAS   | CODMUNRES   | ANO_OBITO   | classificacao       |
|----+------------+-------------+-------------+---------------------|
|  0 | I219       | _292740     | _2019       | Avoidable mortality |
|  1 | I10        | _355100     | _2019       | Avoidable mortality |
|  2 | X700       | _500630     | _2019       | Avoidable mortality |
|  3 | C539       | _500500     | _2019       | causa não evitável  |
|  4 | V220       | _500627     | _2019       | Avoidable mortality |
+----+------------+-------------+-------------+---------------------+


In [83]:
# Mostrar resultados de classificacao
dobr_evit_merged_temp01.groupby('classificacao').count()

,CAUSABAS,CODMUNRES,ANO_OBITO
classificacao,,,
Avoidable mortality,2537888,2537888,2537888
causa não evitável,1909034,1909034,1909034


In [84]:
# Manter na base só dados de CID evitáveis
dobr_evit_merged_temp02 = dobr_evit_merged_temp01[dobr_evit_merged_temp01['classificacao']=='Avoidable mortality']

# mostrar dados gerais da tabela após transformações
tab = dobr_evit_merged_temp02
print(f'Registro da tabela : {tab.shape[0]}')
print(tabulate(tab.head(), headers='keys', tablefmt='psql'))

Registro da tabela : 2537888
+----+------------+-------------+-------------+---------------------+
|    | CAUSABAS   | CODMUNRES   | ANO_OBITO   | classificacao       |
|----+------------+-------------+-------------+---------------------|
|  0 | I219       | _292740     | _2019       | Avoidable mortality |
|  1 | I10        | _355100     | _2019       | Avoidable mortality |
|  2 | X700       | _500630     | _2019       | Avoidable mortality |
|  4 | V220       | _500627     | _2019       | Avoidable mortality |
|  5 | W019       | _500325     | _2019       | Avoidable mortality |
+----+------------+-------------+-------------+---------------------+


#### Resultados do merge de cid_avoid e dobr

In [85]:
# Salvar arquivo final de DOBR
dobr_evit_merged_temp02.to_parquet('files_clean/dobr_evit_merged_v2.parquet')

In [87]:
# Contador

shape_origin = dobr_evit_merged_temp.CAUSABAS.count()
shape_filt = dobr_evit_merged_temp02.CAUSABAS.count()
dif = shape_origin - shape_filt


print_y(f" O número de óbitos após a limpeza e que constituía a base de dados de DO era de {shape_origin} óbitos ")
print_y(f" O número de registros excluídos por apresentar CIDs NÃO EVITÁVEIS foi de {dif} óbitos ")
print_y(f" O número de óbitos por CID EVITÁVEIS é de {shape_filt} óbitos ")

 ==>  O número de óbitos após a limpeza e que constituía a base de dados de DO era de 4446922 óbitos 
 ==>  O número de registros excluídos por apresentar CIDs NÃO EVITÁVEIS foi de 1909034 óbitos 
 ==>  O número de óbitos por CID EVITÁVEIS é de 2537888 óbitos 


### Merge de dados de Municípios ao arquivo principal do SIM

Serão utilizados os arquivos:

* população: 'files_clean/censo_pop_clean.csv'

* sim: 'files_clean/dobr_clean.csv'

Como a população está clivada em Município, Idade (previamente, todos abaixo de 75 anos), Sexo e Raça, estes campos serão utilizados de maneira conjunta para realizar o merge.

In [88]:
# Verificar colunas de população no arquivo censo_pop_clean
censo_pop_mun_4merge_temp = pd.read_parquet('files_clean/censo_pop_mun.parquet')

# mostrar dados gerais da tabela após transformações
tab = censo_pop_mun_4merge_temp
print(f'Registro da tabela : {tab.shape[0]}')
print(tabulate(tab.head(), headers='keys', tablefmt='psql'))

Registro da tabela : 11140
+----+-------+---------+------+---------------+----------------------------+-------------+---------------+
|    |   Ano | uf_co   | UF   | mun_cod_sim   | mun_no                     |   pop_total |   pop_under74 |
|----+-------+---------+------+---------------+----------------------------+-------------+---------------|
|  0 |  2010 | _11     | RO   | _110001       | Alta Floresta D'Oeste (RO) |       24392 |         23948 |
|  1 |  2010 | _11     | RO   | _110002       | Ariquemes (RO)             |       90353 |         89037 |
|  2 |  2010 | _11     | RO   | _110003       | Cabixi (RO)                |        6313 |          6180 |
|  3 |  2010 | _11     | RO   | _110004       | Cacoal (RO)                |       78574 |         76947 |
|  4 |  2010 | _11     | RO   | _110005       | Cerejeiras (RO)            |       17029 |         16646 |
+----+-------+---------+------+---------------+----------------------------+-------------+---------------+


In [89]:
# Criar um campo string que concatene ANO, CODMUN_SIM, Sexo e cor
censo_pop_mun_4merge_temp01 = censo_pop_mun_4merge_temp.copy()

censo_pop_mun_4merge_temp01['POP_MERGE'] = '_' + censo_pop_mun_4merge_temp01['Ano'].astype(str) + censo_pop_mun_4merge_temp01['mun_cod_sim']


# mostrar dados gerais da tabela após transformações
tab = censo_pop_mun_4merge_temp01
print(f'Registro da tabela : {tab.shape[0]}')
print(tabulate(tab.head(), headers='keys', tablefmt='psql'))

Registro da tabela : 11140
+----+-------+---------+------+---------------+----------------------------+-------------+---------------+--------------+
|    |   Ano | uf_co   | UF   | mun_cod_sim   | mun_no                     |   pop_total |   pop_under74 | POP_MERGE    |
|----+-------+---------+------+---------------+----------------------------+-------------+---------------+--------------|
|  0 |  2010 | _11     | RO   | _110001       | Alta Floresta D'Oeste (RO) |       24392 |         23948 | _2010_110001 |
|  1 |  2010 | _11     | RO   | _110002       | Ariquemes (RO)             |       90353 |         89037 | _2010_110002 |
|  2 |  2010 | _11     | RO   | _110003       | Cabixi (RO)                |        6313 |          6180 | _2010_110003 |
|  3 |  2010 | _11     | RO   | _110004       | Cacoal (RO)                |       78574 |         76947 | _2010_110004 |
|  4 |  2010 | _11     | RO   | _110005       | Cerejeiras (RO)            |       17029 |         16646 | _2010_110005

In [90]:
# Contar quantos valores são repetidos na coluna 'POP_MERGE'
repeated_values = censo_pop_mun_4merge_temp01['POP_MERGE'].value_counts()

# Filtrar os valores que aparecem mais de uma vez
repeated_more_than_once = repeated_values[repeated_values > 1]

# Exibir os valores repetidos
print(repeated_more_than_once)

Series([], Name: count, dtype: int64)


In [91]:
# Exibir arquivo dobr_clean
dobr_merged_temp = pd.read_parquet('files_clean/dobr_evit_merged_v2.parquet')

# mostrar dados gerais da tabela após transformações
tab = dobr_merged_temp
print(f'Registro da tabela : {tab.shape[0]}')
print(tabulate(tab.head(), headers='keys', tablefmt='psql'))

Registro da tabela : 2537888
+----+------------+-------------+-------------+---------------------+
|    | CAUSABAS   | CODMUNRES   | ANO_OBITO   | classificacao       |
|----+------------+-------------+-------------+---------------------|
|  0 | I219       | _292740     | _2019       | Avoidable mortality |
|  1 | I10        | _355100     | _2019       | Avoidable mortality |
|  2 | X700       | _500630     | _2019       | Avoidable mortality |
|  4 | V220       | _500627     | _2019       | Avoidable mortality |
|  5 | W019       | _500325     | _2019       | Avoidable mortality |
+----+------------+-------------+-------------+---------------------+


In [92]:
# Gerar um campo de ano para proxy de 2022 (para ser aplicados aos anos de 19 a 22)
dobr_merged_temp01 = dobr_merged_temp.copy()

dobr_merged_temp01['ANO_PROXY'] = dobr_merged_temp01['ANO_OBITO'].apply(lambda x: '_2010' if x == '_2010' else '_2022')

# Criar um campo string que concatene ANO, CODMUNRES
dobr_merged_temp01['POP_MERGE'] = dobr_merged_temp01['ANO_PROXY'].astype(str) + dobr_merged_temp01['CODMUNRES']

# mostrar dados gerais da tabela após transformações
tab = dobr_merged_temp01
print(f'Registro da tabela : {tab.shape[0]}')
print(tabulate(tab.head(), headers='keys', tablefmt='psql'))

Registro da tabela : 2537888
+----+------------+-------------+-------------+---------------------+-------------+--------------+
|    | CAUSABAS   | CODMUNRES   | ANO_OBITO   | classificacao       | ANO_PROXY   | POP_MERGE    |
|----+------------+-------------+-------------+---------------------+-------------+--------------|
|  0 | I219       | _292740     | _2019       | Avoidable mortality | _2022       | _2022_292740 |
|  1 | I10        | _355100     | _2019       | Avoidable mortality | _2022       | _2022_355100 |
|  2 | X700       | _500630     | _2019       | Avoidable mortality | _2022       | _2022_500630 |
|  4 | V220       | _500627     | _2019       | Avoidable mortality | _2022       | _2022_500627 |
|  5 | W019       | _500325     | _2019       | Avoidable mortality | _2022       | _2022_500325 |
+----+------------+-------------+-------------+---------------------+-------------+--------------+


In [93]:
# Verificar operação acima
print(f'Anos de ANO_OBITO {dobr_merged_temp01['ANO_OBITO'].unique()}')
print(f'Anos de ANO_PROXY {dobr_merged_temp01['ANO_PROXY'].unique()}')

Anos de ANO_OBITO ['_2019' '_2020' '_2021' '_2022' '_2010']
Anos de ANO_PROXY ['_2022' '_2010']


########## Este bloco de código a seguir retira o CAUSABAS ##########

In [94]:
# Agrupar por POP_MERGE e somar os registros, não contabilizando mais por CAUSABAS

col_group= [
            # 'CAUSABAS', 
            'CODMUNRES', 'ANO_OBITO','POP_MERGE']
dobr_merged_temp02 = dobr_merged_temp01.groupby(col_group)['CAUSABAS'].count().reset_index()

# Renomear o campo CAUSABAS para CONTADOR
dobr_merged_temp02.rename(columns={'CAUSABAS':'CONTADOR'}, inplace=True)


# mostrar dados gerais da tabela após transformações
tab = dobr_merged_temp02
print(f'Registros da tabela : {tab.shape[0]}')
print(f'Total de óbitos : {tab.CONTADOR.sum()}')
print(tabulate(tab.head(), headers='keys', tablefmt='psql'))

Registros da tabela : 27923
Total de óbitos : 2537888
+----+-------------+-------------+--------------+------------+
|    | CODMUNRES   | ANO_OBITO   | POP_MERGE    |   CONTADOR |
|----+-------------+-------------+--------------+------------|
|  0 | _110000     | _2010       | _2010_110000 |         20 |
|  1 | _110000     | _2019       | _2022_110000 |          9 |
|  2 | _110000     | _2020       | _2022_110000 |          7 |
|  3 | _110000     | _2021       | _2022_110000 |          4 |
|  4 | _110000     | _2022       | _2022_110000 |          6 |
+----+-------------+-------------+--------------+------------+


In [95]:
# Verificação dos nomes das colunas
censo_pop_mun_4merge_temp01.columns

Index(['Ano', 'uf_co', 'UF', 'mun_cod_sim', 'mun_no', 'pop_total',
       'pop_under74', 'POP_MERGE'],
      dtype='object')

In [96]:
# Definir campos de censo_pop para merge
censo_pop_mun_4merge_temp02 = censo_pop_mun_4merge_temp01[['POP_MERGE','pop_under74','UF','mun_no']]

# Realizar o merge dos dataframes
dobr_merged_temp03 = pd.merge(dobr_merged_temp02, censo_pop_mun_4merge_temp02, left_on=['POP_MERGE'], right_on=['POP_MERGE'], how='left')

# mostrar dados gerais da tabela após transformações
tab = dobr_merged_temp03
print(f'Registros da tabela : {tab.shape[0]}')
print(f'Total de óbitos : {tab.CONTADOR.sum()}')
print(tabulate(tab.head(), headers='keys', tablefmt='psql'))

Registros da tabela : 27923
Total de óbitos : 2537888
+----+-------------+-------------+--------------+------------+---------------+------+----------+
|    | CODMUNRES   | ANO_OBITO   | POP_MERGE    |   CONTADOR |   pop_under74 |   UF |   mun_no |
|----+-------------+-------------+--------------+------------+---------------+------+----------|
|  0 | _110000     | _2010       | _2010_110000 |         20 |           nan |  nan |      nan |
|  1 | _110000     | _2019       | _2022_110000 |          9 |           nan |  nan |      nan |
|  2 | _110000     | _2020       | _2022_110000 |          7 |           nan |  nan |      nan |
|  3 | _110000     | _2021       | _2022_110000 |          4 |           nan |  nan |      nan |
|  4 | _110000     | _2022       | _2022_110000 |          6 |           nan |  nan |      nan |
+----+-------------+-------------+--------------+------------+---------------+------+----------+


In [97]:
# Filtrar registros onde pop_under74 não é nulo
dobr_merged_temp04 = dobr_merged_temp03.dropna(subset=['pop_under74']).reset_index(drop=True)

# mostrar dados gerais da tabela após transformações
tab = dobr_merged_temp04
print(f'Registros da tabela : {tab.shape[0]}')
print(f'Total de óbitos : {tab.CONTADOR.sum()}')
print(tabulate(tab.head(), headers='keys', tablefmt='psql'))

Registros da tabela : 27817
Total de óbitos : 2533528
+----+-------------+-------------+--------------+------------+---------------+------+----------------------------+
|    | CODMUNRES   | ANO_OBITO   | POP_MERGE    |   CONTADOR |   pop_under74 | UF   | mun_no                     |
|----+-------------+-------------+--------------+------------+---------------+------+----------------------------|
|  0 | _110001     | _2010       | _2010_110001 |         58 |         23948 | RO   | Alta Floresta D'Oeste (RO) |
|  1 | _110001     | _2019       | _2022_110001 |         67 |         20742 | RO   | Alta Floresta D'Oeste (RO) |
|  2 | _110001     | _2020       | _2022_110001 |         68 |         20742 | RO   | Alta Floresta D'Oeste (RO) |
|  3 | _110001     | _2021       | _2022_110001 |         45 |         20742 | RO   | Alta Floresta D'Oeste (RO) |
|  4 | _110001     | _2022       | _2022_110001 |         64 |         20742 | RO   | Alta Floresta D'Oeste (RO) |
+----+-------------+------

##### Arquivo tratado final

In [98]:
dobr_merged_temp04.to_parquet('files_clean/dobr_merged_v2.parquet')

In [100]:
# Resultados

obito_origin = dobr_merged_temp.CAUSABAS.count()
obito_final = dobr_merged_temp04.CONTADOR.sum()
dif = obito_origin - obito_final


print_y(f" O número de óbitos após merge com tabela de CIDs evitáveis era de {obito_origin} óbitos ")
print_y(f" Foram excluídos, por apresentar problemas para carga de informações de município, {dif} óbitos ")
print_y(f" O número de óbitos total após filtro de CID evitáveis e inclusão de população do censo é de {obito_final} óbitos ")


# # Filtrar registros onde pop_under74 é null
# dobr_merged_pop_nulos = dobr_merged_pop[dobr_merged_pop['pop_under74'].isnull()]

# # Exibir os registros filtrados
# dobr_merged_pop_nulos['obito_count'].sum()

 ==>  O número de óbitos após merge com tabela de CIDs evitáveis era de 2537888 óbitos 
 ==>  Foram excluídos, por apresentar problemas para carga de informações de município, 4360 óbitos 
 ==>  O número de óbitos total após filtro de CID evitáveis e inclusão de população do censo é de 2533528 óbitos 


# Etapa 3 - Aplicação de Modelo do HEAT ao Bloco

O modelo HEAT apresenta um framework bem rígido e objetivo para carga de dados.

O dataset precisa estar organizado com os labels e de acordo com a definição presente no template.

E depois precisa ser validado em planilha excel disponibilizada pela organização.

Ajustes: com a identificação da população por sexo e raça, encontramos 37 registros com população 0. Tivemos que excluir estes registros (incluindo os óbitos) no momento da preparação do dataset do heat em razão do modelo não aceitar estes valores.

## Dataset 2 - BR/uf/Mun


O primeiro dataset que iremos construir tem as seguintes características:

* setting: Brasil
* dimension: UF
* subgroup: município

Estas escolhas implicam em podermos disponibilizar ao usuário do HEAT a possibilidade de ver os municípios (dimensões) de maneira agregada por UF. 


In [3]:
# Define the URL and the destination path
url = "https://srhdpeuwpubsa.blob.core.windows.net/whdh/HIDR/heat_plus_template_validation.xlsm"
destination_path = "downloads_outros/heat_plus_template_validation.xlsm"

# Download the file
response = requests.get(url)
with open(destination_path, 'wb') as file:
    file.write(response.content)

# Note: Enabling macros programmatically is not recommended due to security risks.
# Macros should be enabled manually in Excel settings.

In [101]:
# Abrir arquivo final de DO após tratamento
heatds_temp = pd.read_parquet('files_clean/dobr_merged_v2.parquet')

# mostrar dados gerais da tabela após transformações
tab = heatds_temp
print(f'Registros da tabela : {tab.shape[0]}')
print(f'Total de óbitos : {tab.CONTADOR.sum()}')
print(tabulate(tab.head(), headers='keys', tablefmt='psql'))

Registros da tabela : 27817
Total de óbitos : 2533528
+----+-------------+-------------+--------------+------------+---------------+------+----------------------------+
|    | CODMUNRES   | ANO_OBITO   | POP_MERGE    |   CONTADOR |   pop_under74 | UF   | mun_no                     |
|----+-------------+-------------+--------------+------------+---------------+------+----------------------------|
|  0 | _110001     | _2010       | _2010_110001 |         58 |         23948 | RO   | Alta Floresta D'Oeste (RO) |
|  1 | _110001     | _2019       | _2022_110001 |         67 |         20742 | RO   | Alta Floresta D'Oeste (RO) |
|  2 | _110001     | _2020       | _2022_110001 |         68 |         20742 | RO   | Alta Floresta D'Oeste (RO) |
|  3 | _110001     | _2021       | _2022_110001 |         45 |         20742 | RO   | Alta Floresta D'Oeste (RO) |
|  4 | _110001     | _2022       | _2022_110001 |         64 |         20742 | RO   | Alta Floresta D'Oeste (RO) |
+----+-------------+------

In [38]:
# # Agregar os dados sem CID e gerar colunas de soma de obito_count e pop_under74
# heat_semCID = heat_.groupby(['UF', 'mun_nome', 'CODMUNRES', 'SEXO', 'RACACOR', 'ANO']).agg({
#     'obito_count': 'sum',
#     'pop_under74': 'sum'
# }).reset_index()

In [102]:
# Gerar o indicador de mortalidade (por 100 mil habitantes) e exibir as primeiras linhas do DataFrame
heatds_temp01 = heatds_temp.copy()

heatds_temp01['mort_ind_mun'] = (heatds_temp01['CONTADOR']/heatds_temp01['pop_under74'])*100000

# mostrar dados gerais da tabela após transformações
tab = heatds_temp01
print(f'Registros da tabela : {tab.shape[0]}')
print(f'Total de óbitos : {tab.CONTADOR.sum()}')
print(tabulate(tab.head(), headers='keys', tablefmt='psql'))

Registros da tabela : 27817
Total de óbitos : 2533528
+----+-------------+-------------+--------------+------------+---------------+------+----------------------------+----------------+
|    | CODMUNRES   | ANO_OBITO   | POP_MERGE    |   CONTADOR |   pop_under74 | UF   | mun_no                     |   mort_ind_mun |
|----+-------------+-------------+--------------+------------+---------------+------+----------------------------+----------------|
|  0 | _110001     | _2010       | _2010_110001 |         58 |         23948 | RO   | Alta Floresta D'Oeste (RO) |        242.191 |
|  1 | _110001     | _2019       | _2022_110001 |         67 |         20742 | RO   | Alta Floresta D'Oeste (RO) |        323.016 |
|  2 | _110001     | _2020       | _2022_110001 |         68 |         20742 | RO   | Alta Floresta D'Oeste (RO) |        327.837 |
|  3 | _110001     | _2021       | _2022_110001 |         45 |         20742 | RO   | Alta Floresta D'Oeste (RO) |        216.951 |
|  4 | _110001     | _

In [105]:
# Contar e gerar tabela de indicador de mortalidade infinitos [inf]
registros_inf = heatds_temp01[np.isinf(heatds_temp01['mort_ind_mun'])]
inf_count = np.isinf(heatds_temp01['mort_ind_mun']).sum()
nulos_mort_ind_mun = heatds_temp01['mort_ind_mun'].isnull().sum()


# Exibir os registros filtrados
print(f"Número de valores infinitos em mort_ind_mun: {inf_count}")
print(f"Valores nulos em 'mort_ind_mun': {nulos_mort_ind_mun}")



# Exportar registros de indicador de mortalidade infinitos [inf]
# registros_inf.to_csv('exportar/relatorio_reg_mort_ind_inf.csv')

Número de valores infinitos em mort_ind_mun: 0
Valores nulos em 'mort_ind_mun': 0


In [ ]:
# # Limpar a base de registros de indicador de mortalidade infinitos [inf]
# heat_ds_treat2 = heat_ds_treat.copy()

# # Substituir valores infinitos por NaN
# heat_ds_treat2['mort_ind_mun'].replace([np.inf, -np.inf], np.nan, inplace=True)

In [ ]:
# # Rodar se for necessário excluir valores nulos
# heatds_temp02 = heatds_temp01.copy()

# # Excluir registros com NaN na coluna mort_ind_mun
# heatds_temp02.dropna(subset=['mort_ind_mun'], inplace=True)

# # Mudar o nome da coluna mort_ind para mort_ind_mun
# # heat_ds_treat2.rename(columns={'mort_ind':'mort_ind_mun'}, inplace=True)


# # Verificar as primeiras linhas do DataFrame atualizado
# heat_ds_treat1.head()

In [106]:
# Verificar se existe valor infinito ou negativo na coluna mort_ind_mun
heatds_temp01.describe()

,CONTADOR,pop_under74,mort_ind_mun
count,27817.000000,2.781700e+04,27817.000000
mean,91.078405,3.464735e+04,268.207663
std,510.470539,1.967627e+05,87.673481
min,1.000000,7.820000e+02,12.640627
25%,13.000000,4.978000e+03,211.782434
50%,27.000000,1.053200e+04,263.208494
75%,62.000000,2.335900e+04,317.028986
max,28299.000000,1.089676e+07,940.733772


##### Agrupar por UF para gerar setting e setting-average

De acordo com o template do HEAT, o setting average: "must be consistent for the same combinantion of: setting, year, source and indicator".

Isto implica que os valores para cada setting (UF) não traz interferência nem da dimensão (município) e nem do subgrupo (sexo-raça-cor).

In [ ]:
# # Agrupar por 'UF', 'ANO', e calcular a média de 'mort_ind'
# dobr_grouped_uf = heat_ds_treat2.groupby(['UF',
#                                              'ANO_OBITO',
#                                             #  'SEXO',
#                                             #  'RACACOR'
#                                              ]).agg({'CONTADOR': 'sum',
#                                                     'pop_under74': 'sum',
#                                                     'mort_ind_mun': 'mean'
#                                                 }).reset_index()
# dobr_grouped_uf.rename(columns={'mort_ind_mun': 'mort_ind_uf',
#                                 'CONTADOR': 'obito_count_uf',
#                                 'pop_under74': 'pop_under74_uf',
#                                 }, inplace=True)


# # Exibir as primeiras linhas do DataFrame resultante
# dobr_grouped_uf.head()

In [ ]:
# dobr_grouped_uf.describe()

In [ ]:
# Salvar arquivo por UF para exportação
# dobr_grouped_uf.to_excel('exportar/dobr_grouped_uf_v2.xlsx', index=False)

In [ ]:
# heat_ds_treat2.columns

In [ ]:
# # Trazer dados por UF (setting e setting_average)

# # Realizar o merge entre heat_ e uf_sett
# heat_treat00 = pd.merge(heat_ds_treat2, dobr_grouped_uf, on=['UF',
#                                                                 'ANO_OBITO',
#                                                                 # 'SEXO',
#                                                                 # 'RACACOR'
#                                                                 ], how='left')

# # Exibir as primeiras linhas do dataframe resultante
# heat_treat00.head()

In [ ]:
# # Teste de aplicação de setting_average (médias por UF)
# filtered_data = heat_treat00[   (heat_treat00['ANO_OBITO'] == '_2022') &
#                                 (heat_treat00['UF'] == 'AC') 
#                                 # & (heat_treat00['SEXO'] == '_1') &
#                                 #(heat_treat00['RACACOR'] == '_4')
#                               ]
# filtered_data.head()

In [ ]:
# heat_treat00.columns

In [ ]:
# heat_treat00.pop_under74_uf.sum()

In [ ]:
# obito_num = heat_treat00.CONTADOR.sum()
# pop_under74 = heat_treat00.pop_under74.sum()

# print(f"\033[33m ==> Número de mortes: {obito_num} \033[0m")
# print(f"\033[33m ==> Número de população abaixo de 75 anos é de {pop_under74} \033[0m")  


In [ ]:
# heat_template = row
# heat_template

##### Criar colunas do dataset

In [115]:
# Na média ponderada de mortalidade evitável, pegamos os valores de mortalidade para cada município e dividimos pela população, para conseguir o valor para o Brasil e por ano

# Função para calcular a média ponderada
def weighted_avg(group):
    return np.average(group['mort_ind_mun'], weights=group['pop_under74'])

# Agrupar os dados pelo ano e aplicar a função de média ponderada
weighted_avg_per_year = heatds_temp01.groupby('ANO_OBITO').apply(weighted_avg).reset_index()

# Renomear as colunas para melhor entendimento
weighted_avg_per_year.columns = ['ANO_OBITO', 'set_avg_BR']

# Exibir o resultado
print(weighted_avg_per_year)

  ANO_OBITO  set_avg_BR
0     _2010  257.615194
1     _2019  258.919766
2     _2020  256.542023
3     _2021  266.310807
4     _2022  274.720874


/tmp/ipykernel_4975/4059653304.py:8: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  weighted_avg_per_year = heatds_temp01.groupby('ANO_OBITO').apply(weighted_avg).reset_index()


In [116]:
# Realizar o merge entre heat_ds_treat2 e grouped_data usando a coluna ANO_OBITO
heatds_temp02 = pd.merge(heatds_temp01, weighted_avg_per_year[['ANO_OBITO', 'set_avg_BR']], on='ANO_OBITO', how='left')

# mostrar dados gerais da tabela após transformações
tab = heatds_temp02
print(f'Registros da tabela : {tab.shape[0]}')
print(f'Total de óbitos : {tab.CONTADOR.sum()}')
print(tabulate(tab.head(), headers='keys', tablefmt='psql'))

Registros da tabela : 27817
Total de óbitos : 2533528
+----+-------------+-------------+--------------+------------+---------------+------+----------------------------+----------------+--------------+
|    | CODMUNRES   | ANO_OBITO   | POP_MERGE    |   CONTADOR |   pop_under74 | UF   | mun_no                     |   mort_ind_mun |   set_avg_BR |
|----+-------------+-------------+--------------+------------+---------------+------+----------------------------+----------------+--------------|
|  0 | _110001     | _2010       | _2010_110001 |         58 |         23948 | RO   | Alta Floresta D'Oeste (RO) |        242.191 |      257.615 |
|  1 | _110001     | _2019       | _2022_110001 |         67 |         20742 | RO   | Alta Floresta D'Oeste (RO) |        323.016 |      258.92  |
|  2 | _110001     | _2020       | _2022_110001 |         68 |         20742 | RO   | Alta Floresta D'Oeste (RO) |        327.837 |      256.542 |
|  3 | _110001     | _2021       | _2022_110001 |         45 |  

In [118]:
# Gerar colunas sem informação ou com informação padronizada
# heat_treat01 = heat_semCID_clean # caso não seja agrupado por UF anteriormente
heatds_temp03 = heatds_temp02.copy() # caso seja agrupado por UF anteriormente

heatds_temp03['setting'] = 'Brasil'
# heat_treat01['setting_average'] = set_avg_BR

heatds_temp03['indicator_abbr'] = 'AVM'
heatds_temp03['indicator_name'] = 'Avoidable mortality (deaths per 100 000 population)'
heatds_temp03['source'] = 'SIM/SUS & Censo/IBGE'
heatds_temp03['favourable_indicator'] = 0
heatds_temp03['indicator_scale'] = 100000
heatds_temp03['ordered_dimension'] = 0
heatds_temp03['subgroup_order'] = 0
heatds_temp03['reference_subgroup'] = 0
heatds_temp03['note'] = ''
heatds_temp03['se'] = ''
heatds_temp03['ci_lb'] = ''
heatds_temp03['ci_ub'] = ''
heatds_temp03['iso3'] = 'BRA'

# Renomear colunas de heat_treat01
heatds_temp03 = heatds_temp03.rename(columns={
    'ANO_OBITO':'date',
    'pop_under74':'population',
    'UF':'dimension',
    'mun_no':'subgroup',
    'mort_ind_mun':'estimate',
    'set_avg_BR':'setting_average',
    # 'mort_ind_uf':'setting_average'
})

# Ajustar coluna de ano
heatds_temp03['date'] = heatds_temp03['date'].str.replace('_','').astype(int)


# mostrar dados gerais da tabela após transformações
tab = heatds_temp03
print(f'Registros da tabela : {tab.shape[0]}')
print(f'Total de óbitos : {tab.CONTADOR.sum()}')
print(tabulate(tab.head(), headers='keys', tablefmt='psql'))


Registros da tabela : 27817
Total de óbitos : 2533528
+----+-------------+--------+--------------+------------+--------------+-------------+----------------------------+------------+-------------------+-----------+------------------+-----------------------------------------------------+----------------------+------------------------+-------------------+---------------------+------------------+----------------------+--------+------+---------+---------+--------+
|    | CODMUNRES   |   date | POP_MERGE    |   CONTADOR |   population | dimension   | subgroup                   |   estimate |   setting_average | setting   | indicator_abbr   | indicator_name                                      | source               |   favourable_indicator |   indicator_scale |   ordered_dimension |   subgroup_order |   reference_subgroup | note   | se   | ci_lb   | ci_ub   | iso3   |
|----+-------------+--------+--------------+------------+--------------+-------------+----------------------------+---------

In [285]:
# # Trazer labels para sexo e raçacor
# heat_treat02 = heat_treat01.copy()

# heat_treat02['SEXO'] = heat_treat02['SEXO'].map({
#                                             '_1':'Homens',
#                                             '_2':'Mulheres'
#                                             })
# heat_treat02['RACACOR'] = heat_treat02['RACACOR'].map({
#                                             '_1':'Branca',
#                                             '_2':'Preta',
#                                             '_3':'Amarela',
#                                             '_4':'Parda',
#                                             '_5':'Indígena',
#                                             '_9':'Sem declaração'
#                                             })

# heat_treat02['subgroup'] = heat_treat02['SEXO'] + ' - raça/cor '  +  heat_treat02['RACACOR']


In [ ]:
# heat_treat02['subgroup'].unique()

In [ ]:
# heat_treat02.columns

In [120]:
# Define the order of columns
column_order = [
    'setting',
    'date',
    'source',
    'indicator_abbr',
    'indicator_name',
    'dimension',
    'subgroup',
    'estimate',
    'se',
    'ci_lb',
    'ci_ub',
    'population',
    'note',
    'setting_average',
    'iso3',
    'favourable_indicator',
    'indicator_scale',
    'ordered_dimension',
    'subgroup_order',
    'reference_subgroup'
    ]  


# Reorder the columns in the DataFrame
heatds_temp04 = heatds_temp03[column_order]


In [123]:
# Transformar estimate e setting_average em int
heatds_temp05 = heatds_temp04.copy()

heatds_temp05['estimate'] = heatds_temp05['estimate'].astype(int)
heatds_temp05['setting_average'] = heatds_temp05['setting_average'].astype(int)

# mostrar dados gerais da tabela após transformações
tab = heatds_temp05
print(f'Registros da tabela : {tab.shape[0]}')
# print(f'Total de óbitos : {tab.CONTADOR.sum()}')
print(tabulate(tab.head(), headers='keys', tablefmt='psql'))

Registros da tabela : 27817
+----+-----------+--------+----------------------+------------------+-----------------------------------------------------+-------------+----------------------------+------------+------+---------+---------+--------------+--------+-------------------+--------+------------------------+-------------------+---------------------+------------------+----------------------+
|    | setting   |   date | source               | indicator_abbr   | indicator_name                                      | dimension   | subgroup                   |   estimate | se   | ci_lb   | ci_ub   |   population | note   |   setting_average | iso3   |   favourable_indicator |   indicator_scale |   ordered_dimension |   subgroup_order |   reference_subgroup |
|----+-----------+--------+----------------------+------------------+-----------------------------------------------------+-------------+----------------------------+------------+------+---------+---------+--------------+--------+----

In [124]:
### Arquivo pronto para exportação
heatds_temp05.to_excel('exportar/ds_4val_v2.xlsx', index=False)

In [4]:
# Gerar arquivo dataset diretamente no arquivo de validação do HEAT

# Carregar o arquivo original
workbook = openpyxl.load_workbook('downloads_outros/heat_plus_template_validation.xlsm', keep_vba=True)

# Apagar a aba 'template'
if 'template' in workbook.sheetnames:
    del workbook['template']

# Carregar o arquivo que será copiado como nova aba
workbook_to_copy = openpyxl.load_workbook('exportar/ds_4val_v2.xlsx')
sheet_to_copy = workbook_to_copy.active

# Copiar a aba para o workbook original
new_sheet = workbook.create_sheet('template')

for row in sheet_to_copy.iter_rows(values_only=True):
    new_sheet.append(row)

# Salvar o arquivo com o novo nome
workbook.save('exportar/heat_plus_template_ds_4val_v2.xlsm')

# Etapa 4 - Reunião de alinhamento - ajustes

# Etapa 5 - Preparação para publicação de resultados